In [76]:
import torch
import torch.nn.functional as F
from torchvision import transforms
import os
import numpy as np
import argparse
import pandas as pd
from PIL import Image

from matplotlib import pyplot as plt

import time

import pytorch_ssim
from  dataset.MVTECDataLoader import MVTECTestDataLoader
from HaloAE import HaloAE_model as ae
import torch.utils.data as data

In [77]:
model = ae(Init = True, n_layer_vgg = 4, first_out_channel = 380, FM = False, num_classes = 2)

EnD make layer features map
Loading vgg 
Loading loaded  
INIT Extractor

Initializing network weights.........


In [78]:
checkpoint_path = '/home/nicerons/Documents/hazelnut/HaloAE_tumor_segm_without_cutPaste_color_augment_5_percent_07_04_.pt'

In [79]:
c_checkpoint_path = checkpoint_path.split('.')[0] + '_' + str(250) + '.pt'
model.load_state_dict(torch.load(c_checkpoint_path), strict = False)
model.eval()

HaloAE_model(
  (feature_extractor): Extractor(
    (feature): VGG19(
      (conv1_1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1))
      (relu1_1): ReLU(inplace=True)
      (conv1_2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
      (relu1_2): ReLU(inplace=True)
      (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (conv2_1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
      (relu2_1): ReLU(inplace=True)
      (conv2_2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1))
      (relu2_2): ReLU(inplace=True)
      (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (conv3_1): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1))
      (relu3_1): ReLU(inplace=True)
      (conv3_2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1))
      (relu3_2): ReLU(inplace=True)
      (conv3_3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1))
      (relu3_3): ReLU(inplace=True)
      (conv3_4): Conv

Try to load only one image so I can clissify it using the model loaded just above

In [80]:
class TestImageDataLoader(data.Dataset):
    def __init__(self,
                 root='/home/nicerons/Documents/KI67_Normal_Tumor_test2',
                transform_process= None):
        
        self.img_size = 256
        self.img_path_list_file = f'{root}/test_images_list.txt'
        with open(self.img_path_list_file, 'r') as f:
            content =  f.readlines()
        self.files_list = []
        for x in content:
            x =  x.strip()
            self.files_list.append(x)
        self.transform = transform_process

    def __getitem__(self,index):
        imgname = self.files_list[index]
        assert self.img_size%2==0, 'Img size should be even'
        img =  Image.open(self.files_list[index])
        if self.transform is not None:
            img_t= self.transform(img)
        return img_t, imgname
        

    def __len__(self):
        return len(self.files_list)

In [81]:
test_transform = transforms.Compose([])
test_transform.transforms.append(transforms.Resize((256,256)))
test_transform.transforms.append(transforms.ToTensor())
test_transform.transforms.append(transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                    std=[0.229, 0.224, 0.225]))



In [82]:
test_data = TestImageDataLoader(root = '/home/nicerons/Documents/KI67_Normal_Tumor_test2', transform_process = test_transform)
test_loader = torch.utils.data.DataLoader(test_data, batch_size = 1, shuffle = False, num_workers = 4, pin_memory = False)


In [83]:
loss_cross_ent = torch.nn.CrossEntropyLoss()
df = pd.DataFrame({'fname':[], 'loss':[], 'Logit1':[], 'Logit2':[]})

In [84]:
with torch.no_grad():
    Losses = []
    cls = []
    MSEFM = []
    MSEIM = []
    SSIMFM = []
    SSIMIM = []
    Labels = []
    Logit1 = []
    Logit2 = []
    exp_label = []
    l = []
    for c, (img, label) in enumerate(test_loader):
        label = label[0]
        logitImg, feature_map, reconsImg, reconsFM = model(img.cuda())
        
        feature_map = feature_map.cuda(0)
        reconsFM = reconsFM.cuda(0)
        loss1FM = F.mse_loss(reconsFM, feature_map)
        loss2FM = 1 - pytorch_ssim.ssim(feature_map, reconsFM)
        
        reconsImg = reconsImg.cuda(0)
        img = img.cuda(0)
        loss1IM = F.mse_loss(img, reconsImg)
        loss2IM = 1 - pytorch_ssim.ssim(img, reconsImg)
        
        label_n = torch.zeros(1)
        label_n  = label_n.cuda(0)
        label_n = label_n.to(torch.long)
        loss_cls = loss_cross_ent(logitImg, label_n)
        feature_map = feature_map.permute(0,2,3,1)
        feature_map = feature_map.cpu().numpy()
        reconsFM = reconsFM.permute(0,2,3,1)
        reconsFM = reconsFM.cpu().numpy()
        
        if label.split('/')[-2] != 'Normal':
            exp_label.append(0)
            label_n = torch.ones(1)
        if label.split('/')[-2] == 'Normal':
            exp_label.append(1)

        reconsImg = reconsImg.permute(0,2,3,1)
        reconsImg = reconsImg.cpu().numpy()
        img = img.permute(0,2,3,1)
        img = img.cpu().numpy()
        loss = 0.25*loss1FM+0.25*loss2FM + 0.25*loss1IM+0.25*loss2IM
        
        cls.append(loss_cls.item())
        MSEFM.append(loss1FM.item())
        SSIMFM.append(loss2FM.item())
        MSEIM.append(loss1IM.item())
        SSIMIM.append(loss2IM.item())
        Losses.append(loss.item())
        Logit1.append(logitImg.cpu().numpy()[0,0])
        Logit2.append(logitImg.cpu().numpy()[0,1])
        Labels.append(label)
        
    df['fname'] = Labels
    df['loss'] = Losses
    df['cls'] = cls
    df['MSEFM'] = MSEFM
    df['SSIMFM'] = SSIMFM
    df['MSEIM'] = MSEFM
    df['SSIMIM'] = SSIMFM
    df['Logit1'] = Logit1
    df['Logit2'] = Logit2
    df['exp_label'] = exp_label
        
        


In [85]:
df.to_csv('/home/nicerons/Documents/KI67_Normal_Tumor_test2/result.csv')